In [2]:
import sys
sys.path.append('..')
import pandas as pd
from src.utils.audio_utils import get_audio_features
import warnings
warnings.filterwarnings('ignore')

In [3]:
csv_path = "../data/chartex_final.csv"

sorting data by popularity

In [ ]:
# # Load the CSV file into a DataFrame
# df = pd.read_csv(csv_path)

# # Drop the 'number_of_videos_last_14days' and 'song_name' columns
# df = df.drop(columns=['Unnamed: 0'])

# # Sort the DataFrame by the 'track_name' column
# sorted_df = df.sort_values(by='total_likes_count', ascending=False)

# # Reset the index of the sorted DataFrame
# sorted_df = sorted_df.reset_index(drop=True)

# # Overwrite the original CSV file with the sorted data
# sorted_df.to_csv(csv_path, index=False)

# # Print a message to confirm that the file has output_directory overwritten
# print(f'The file {csv_path} has been overwritten with the sorted data.')

getting Spotify data for most popular song

In [12]:
# Load the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Get the first row of data
example_track_data = df.iloc[0]

# Print the resulting row
print(example_track_data)

track_name                        Love You So
track_pop                                   0
artist               The King Khan & BBQ Show
artist_pop                                 37
album                The King Khan & BBQ Show
danceability                            0.389
energy                                  0.896
key                                       5.0
loudness                               -2.622
mode                                      1.0
speechiness                            0.0599
acousticness                             0.79
instrumentalness                      0.00436
liveness                                0.501
valence                                 0.653
tempo                                 115.143
id                     4msYRkezQgynuZNubvVbHk
duration_ms                          225240.0
time_signature                            4.0
artist_name          The King Khan & BBQ Show
total_likes_count                  4187998484
number_of_videos                  

extracting audio features for example song

In [ ]:
audio_dir = "../data/track_downloads"

# Get the additional audio features for the track
features = get_audio_features(example_track_data, audio_dir)

extracting features for all songs

In [ ]:
def batch_extract(df, start_index, end_index):
    for i in range(start_index, end_index):
        print(f'Processing track {i+1} of {len(df)}')
        track_data = df.iloc[i]
        features = get_audio_features(track_data)
        if not features:
            print(f'No features found for track {i+start_index} of {len(df)}, skipping...') 
            continue
        # add features to dataframe
        for feature_names in features.keys():
            df.loc[i, feature_names] = features[feature_names]
        # save dataframe  
        df.to_csv("../data/audio_features.csv", index=False)
    

#for i in range(0, len(df), 10):
#    batch_extract(df, i, i+10)
    

We will now check if all songs were downloaded:

In [5]:
import os
downloaded_songs = os.listdir('../data/track_downloads/')
num_of_songs = len(df.index)
print("All songs downloaded" if len(downloaded_songs)==num_of_songs else "There are missing songs")

There are missing songs


In [ ]:
for i in range(num_of_songs):
    curr_song = df.iloc[i]
    if not curr_song['id'] + '.wav' in downloaded_songs:
        batch_extract(df,i,i+1)

We could not download most of those missing songs.

Next, we will convert the audio files from MP4 format to WAV format in order to be able to use it in the torchaudio library:

In [13]:
import os

os.mkdir('../data/audio')

In [15]:
import subprocess
from tqdm import tqdm

for song in tqdm(downloaded_songs):
    song_path = '../data/track_downloads/' + song
    conv_song_path = '../data/audio/' + song[:-4] + ".mp3"
    ffmpeg_command = f"ffmpeg -i {song_path} -vn -acodec libmp3lame -q:a 4 -ar 22050 {conv_song_path}"

    subprocess.run(ffmpeg_command, shell=True)

100%|██████████| 3920/3920 [4:09:50<00:00,  3.82s/it]    


In [7]:
len(df)

3932

and now let's finish by creating the .csv of the dataset, using the threshold of $5e5$:

In [13]:
converted_songs = [song[:-4] for song in os.listdir('../data/audio/')]


df['viral'] = (df['number_of_videos'] > 5e5).astype('int32')
df.drop(['track_name', 'track_pop', 'artist', 'artist_pop', 'album','number_of_videos'\
        'time_signature', 'artist_name','total_likes_count', 'number_of_videos',\
        'chroma_stft', 'rmse','spec_cent', 'spec_bw', 'rolloff', 'zcr', 'mfcc'], axis=1, errors = 'ignore',inplace=True)

df = df[df['id'].isin(converted_songs)]

In [16]:
df.to_csv("../data/metadata.csv")

# Garbage:

In [8]:
# WAV is not compress and thus weight 170 GB on my disk... Here I try to convert the files to mp3 from the original mp4 but it took a lot of time so I give up for now.
import subprocess

song_path = '../data/track_downloads/' + '00QH0dk4fdur44J1lfbhqv.wav'
conv_song_path = '../data/conv_audio.mp3'
ffmpeg_command = f"ffmpeg -i {song_path} -vn -acodec libmp3lame -q:a 4 -ar 22050 {conv_song_path}"

subprocess.run(ffmpeg_command, shell=True)

CompletedProcess(args='ffmpeg -i ../data/track_downloads/00QH0dk4fdur44J1lfbhqv.wav -vn -acodec libmp3lame -q:a 4 -ar 22050 ../data/conv_audio.mp3', returncode=0)

In [7]:
import subprocess
song = '00QH0dk4fdur44J1lfbhqv.wav'
song_path = '../data/track_downloads/' + song
conv_song_path = '../data/' + song
ffmpeg_command = f"ffmpeg -i {song_path} -acodec pcm_s16le -ar 22050 {conv_song_path}"
print(subprocess.run(ffmpeg_command, shell=True))

CompletedProcess(args='ffmpeg -i ../data/track_downloads/00QH0dk4fdur44J1lfbhqv.wav -acodec pcm_s16le -ar 22050 ../data/00QH0dk4fdur44J1lfbhqv.wav', returncode=0)
